In [1]:
library(igraph)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



## Question 16

In [21]:
#read the graph
filePath = "/Users/ht/desktop/EE232E/proj_2/facebook_combined.txt"
g = read.graph(file = filePath, format="ncol", directed=FALSE)
#plot(g, vertex.label=NA, vertex.size=5,  edge.arrow.size=.2)
g_415 = induced_subgraph(g, c(neighbors(g,V(g)[V(g)$name=="414"]),V(g)[V(g)$name=="414"]))

In [22]:
list_user <- V(g_415)[degree(g_415)==24]$name
print(list_user)
#number of users
print(length(list_user))
users <- c()
for(u in list_user){
    users <- c(users, V(g_415)[V(g_415)$name==u])
}

 [1] "496" "578" "600" "615" "618" "627" "643" "658" "659" "661" "662"
[1] 11


## Question 17

In [23]:
g_415_backup <- g_415
#create 3 measures
common_neighbor <- function(x,y){
    length(intersect(x,y))
}
jaccard <- function(x,y){
    length(intersect(x,y))/length(union(x,y))
}
adamic_adar <- function(x,y){
    count_neighbor <- c()
    for (inter in intersect(x,y)){
        count_neighbor <- c(count_neighbor, 1/log(length(neighbors(g_415, inter))))
    }
    sum(count_neighbor)
}

accuracy <- function(R,P){
    length(intersect(P,R))/length(R)
}

In [27]:
acc_tot_com <- c()
acc_tot_jac <- c()
acc_tot_ada <- c()
for (idx in 1:length(users)){
    user <- users[idx]
    acc_epoch_com <- c()
    acc_epoch_jac <- c()
    acc_epoch_ada <- c()
    for (i in 1:10){
        g_415 <- g_415_backup
        neighbor <- neighbors(g_415,user)
        #create R
        R_i <- c()
        for (idx_nei in 1:length(neighbor)){
            random_sample <- sample(1:100, 1)
            if (random_sample <= 25){
                R_i <- c(R_i, neighbor[idx_nei])
                g_415 <- g_415 %>%
                    delete.edges(E(g_415)[get.edge.ids(g_415, c(user, neighbor[idx_nei]))])
            }
        }
        len_Ri <- length(R_i)
        
        #compute the similarity
        com_score <- c()
        jac_score <- c()
        ada_score <- c()
        candidate_list <- c()
        for(idx_candidate in 1:length(V(g_415))){
            candidate <- V(g_415)[idx_candidate]
            if(length(intersect(candidate, neighbors(g_415, user))) == 0 & candidate != user){
                candidate_list <- c(candidate_list, candidate)
                com_score <- c(com_score, common_neighbor(neighbors(g_415, candidate),neighbors(g_415, user)))
                jac_score <- c(jac_score, jaccard(neighbors(g_415, candidate),neighbors(g_415, user)))
                ada_score <- c(ada_score, adamic_adar(neighbors(g_415, candidate),neighbors(g_415, user)))
            }
        }
        #common neighbor
        com_sort <- sort(com_score,decreasing=TRUE, index.return=TRUE)
        recom_com <- candidate_list[com_sort$ix[1:len_Ri]]
        acc_epoch_com <- c(acc_epoch_com, accuracy(R_i, recom_com))
        
        #jaccard
        jac_sort <- sort(jac_score,decreasing=TRUE, index.return=TRUE)
        recom_jac <- candidate_list[jac_sort$ix[1:len_Ri]]
        acc_epoch_jac <- c(acc_epoch_jac, accuracy(R_i, recom_jac))
        
        #adamic adar
        ada_sort <- sort(ada_score,decreasing=TRUE, index.return=TRUE)
        recom_ada <- candidate_list[ada_sort$ix[1:len_Ri]]
        acc_epoch_ada <- c(acc_epoch_ada, accuracy(R_i, recom_ada))
    }
    acc_tot_com <- c(acc_tot_com, sum(acc_epoch_com)/length(acc_epoch_com))
    acc_tot_jac <- c(acc_tot_jac, sum(acc_epoch_jac)/length(acc_epoch_jac))
    acc_tot_ada <- c(acc_tot_ada, sum(acc_epoch_ada)/length(acc_epoch_ada))
}
print(sum(acc_tot_com)/length(acc_tot_com))
print(sum(acc_tot_jac)/length(acc_tot_jac))
print(sum(acc_tot_ada)/length(acc_tot_ada))

[1] 0.8498377
[1] 0.8208333
[1] 0.8458333


In [28]:
print(acc_tot_com)
print(acc_tot_jac) 
print(acc_tot_ada)

 [1] 0.3894048 1.0000000 0.9264286 0.8301587 0.4133333 1.0000000 0.8952778
 [8] 0.9875000 0.9833333 0.9427778 0.9800000
 [1] 0.1813095 0.9633333 0.9264286 0.8301587 0.5642857 0.9633333 0.8809921
 [8] 0.9341667 0.9323810 0.8927778 0.9600000
 [1] 0.3620238 0.9833333 0.9264286 0.8301587 0.4133333 1.0000000 0.8952778
 [8] 0.9875000 0.9833333 0.9427778 0.9800000


## Question 18

In [2]:
setwd("/Users/ht/desktop/EE232E/proj_2/")
#read the data
file_names = list.files("gplus/")
file_ids = sub("^([^.]*).*", "\\1", file_names)
ego_node_ids = unique(file_ids)
total_percent = c()
ids_circles = c()
#store id which has more than 2 circles
for (id in ego_node_ids) {
    circles_file = paste("gplus/" , id , ".circles" , sep="")
    circles_temp = file(circles_file , open="r")
    circles_content = readLines(circles_temp)
    close(circles_temp)
    
    if(length(circles_content) > 2)
    ids_circles = c(ids_circles, id)
}

In [3]:
print(length(ids_circles))

[1] 57


In [4]:
print(ids_circles)

 [1] "100535338638690515335" "100962871525684315897" "101130571432010257170"
 [4] "101185748996927059931" "101263615503715477581" "101373961279443806744"
 [7] "101541879642294398860" "101626577406833098387" "102170431816592344972"
[10] "102615863344410467759" "102778563580121606331" "103236949470535942612"
[13] "103892332449873403244" "104105354262797387583" "104607825525972194062"
[16] "104672614700283598130" "104987932455782713675" "106186407539128840569"
[19] "106228758905254036967" "106382433884876652170" "106837574755355833243"
[22] "107040353898400532534" "107203023379915799071" "107223200089245371832"
[25] "107459220492917008623" "107489144252174167638" "108883879052307976051"
[28] "109327480479767108490" "109596373340495798827" "110538600381916983600"
[31] "110614416163543421878" "110701307803962595019" "110809308822849680310"
[34] "110971010308065250763" "111048918866742956374" "111091089527727420853"
[37] "112317819390625199896" "112724573277710080670" "113112256846010263985"

## Question 19 & 20

In [4]:
# Plot
plott <-function(g, wc){
layout <-layout.fruchterman.reingold(g)
plot(wc, g, layout=layout, vertex.label=NA, vertex.size=5,  edge.arrow.size=.2)
}

In [5]:
target_ids = c("109327480479767108490","115625564993990145546","101373961279443806744")

In [6]:
mod_list <- c()
for (id in  target_ids) {
    #extract edges and circles
    edges_file = paste("gplus/" , id  , ".edges" , sep="")
    circles_file = paste("gplus/" , id , ".circles" , sep="")
    
    circles_temp = file(circles_file , open="r")
    circles_content = readLines(circles_temp)
    
    circles = list()
    for (i in 1:length(circles_content)) {
        circle_nodes = strsplit(circles_content[i],"\t")
        circles = c(circles, list(circle_nodes[[1]][-1]))
    }
    
    close(circles_temp)
    
    g = read.graph(edges_file , format = "ncol" , directed=TRUE)
    g = add.vertices(g, nv = 1, name = id)
    ego_node_index = which(V(g)$name==id)
    
    add_edge_list = c()
    for (vertex in 1:(vcount(g) - 1)) {
        add_edge_list = c(add_edge_list, c(ego_node_index, vertex))
    }
    
    g = add_edges(g, add_edge_list)
   
    hist(degree(g,mode = "in"))
    hist(degree(g,mode = "out"))
    
    wt_comm <- walktrap.community(g)
    
    plott(g, wt_comm)
    mod_list <- c(mod_list, modularity(wt_comm))
}

print(mod_list)

ERROR: Error: no se pudo encontrar la función "read.graph"


## Question 21&22

In [40]:
h_list <- c()
c_list <- c()
for (id in  target_ids[1]) {
    #extract edges and circles
    edges_file = paste("gplus/" , id  , ".edges" , sep="")
    circles_file = paste("gplus/" , id , ".circles" , sep="")
    
    circles_temp = file(circles_file , open="r")
    circles_content = readLines(circles_temp) #each line represents a circle
    
    circles = list()
    for (i in 1:length(circles_content)) {
        circle_nodes = strsplit(circles_content[i],"\t")
        circles = c(circles, list(circle_nodes[[1]][-1]))
    }
    close(circles_temp)
    
    #entropy of circle
    a <- c()
    total_circle <- c()
    for (cir_id in 1:length(circles)){
        a <- c(a, length(circles[[cir_id]]))
        total_circle <- c(total_circle, unique(circles[[cir_id]]))
    }
    total_circle <- unique(total_circle)
    N = length(total_circle)
    
    H_C = -1*sum(a/N*log((a+0.0001)/N))
    
    #build network
    g = read.graph(edges_file , format = "ncol" , directed=TRUE)
    g = add.vertices(g, nv = 1, name = id)
    ego_node_index = which(V(g)$name==id)
    
    add_edge_list = c()
    for (vertex in 1:(vcount(g) - 1)) {
        add_edge_list = c(add_edge_list, c(ego_node_index, vertex))
    }   
    g <- add_edges(g, add_edge_list)    
    wt_comm <- walktrap.community(g)
    
    #entropy of community
    b <- c()
    for (com_id in 1:length(wt_comm)){
        b <- c(b, length(intersect(wt_comm[[com_id]], total_circle)))
    }
    H_K = -1*sum(b/N*log((b+0.0001)/N))
    
    #conditional entropy
    h.c_k <- c()
    C <- c()
    for (com_id in 1:length(wt_comm)){
        for (cir_id in 1:length(circles)){
            C_ji <- length(intersect(circles[[cir_id]], wt_comm[[com_id]]))
            C <- c(C, C_ji)
            h.c_k <- c(h.c_k, C_ji/N*log((C_ji+0.0001)/(b[com_id]+0.0001)))
        }
    }
    
    H.c_k <- -1*sum(h.c_k)
    h = 1 - H.c_k/H_C
    
    h.k_c <- c()
    C <- c()
    for (cir_id in 1:length(circles)){
        for (com_id in 1:length(wt_comm)){
            C_ji <- length(intersect(circles[[cir_id]], wt_comm[[com_id]]))
            C <- c(C, C_ji)
            h.k_c <- c(h.k_c, C_ji/N*log((C_ji+0.0001)/(a[cir_id]+0.0001)))
        }
    }
    H.k_c <- -1*sum(h.k_c)
    c = 1 - H.k_c/H_K
    
    h_list <- c(h_list, h)
    c_list <- c(c_list, c)
}

[1] "a"
[1] 330 346 419
[1] "total length"
[1] 1095
[1] "unique length"
[1] 764
[1] "H_C"
[1] 1.050779
[1] "b"
[1] 279 397  75  13
[1] "H_K"
[1] 1.005208
[1] "C_ji"
 [1] 251 267  13   4   4 393  75  75   0   0   0  13
[1] "h(c|k)"
 [1] -0.034745366 -0.015364070 -0.052174498 -0.024071294 -0.024071294
 [6] -0.005209136  0.000000000  0.000000000  0.000000000  0.000000000
[11]  0.000000000  0.000000000
[1] "H(c|k)"
[1] 0.1556357
[1] "h"
[1] 0.8518854
[1] "C_ji"
 [1] 251   4  75   0 267   4  75   0  13 393   0  13
[1] "h(k|c)"
 [1] -0.08989993 -0.02310353 -0.14544537  0.00000000 -0.09058081 -0.02335141
 [7] -0.15009322  0.00000000 -0.05909409 -0.03295299  0.00000000 -0.05909409
[1] "H(k|c)"
[1] 0.6736154
[1] "c"
[1] 0.3298743


In [12]:
print(h_list)
print(c_list)

[1] 0.85188544 0.45189098 0.00386597
[1]  0.3298743 -3.4239577 -1.5042390
